<a href="https://colab.research.google.com/gist/TothAron/b0b9687be8f7beda8edb57ce03ce9c0a/transfer_learning_inceptionv3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# To download specific amount of data from Google Open Image Dataset, we need an additional tool
!git clone https://github.com/EscVM/OIDv4_ToolKit.git #download
!pip3 install -r /content/OIDv4_ToolKit/requirements.txt #install requirements

Cloning into 'OIDv4_ToolKit'...
remote: Enumerating objects: 422, done.
remote: Total 422 (delta 0), reused 0 (delta 0), pack-reused 422
Receiving objects: 100% (422/422), 34.08 MiB | 23.90 MiB/s, done.
Resolving deltas: 100% (146/146), done.
     |████████████████████████████████| 3.8 MB 5.2 MB/s 
     |████████████████████████████████| 79 kB 6.2 MB/s 
     |████████████████████████████████| 8.2 MB 37.5 MB/s 
     |████████████████████████████████| 547 kB 43.1 MB/s 
     |████████████████████████████████| 138 kB 46.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: docutils
    Found existing installation: docutils 0.17.1
    Uninstalling docutils-0.17.1:
      Successfully uninstalled docutils-0.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the follo

In [2]:
# get the data 
!python /content/OIDv4_ToolKit/main.py downloader --classes Man --type_csv train --limit 200 --noLabels
!python /content/OIDv4_ToolKit/main.py downloader --classes Cat --type_csv train --limit 200 --noLabels
!python /content/OIDv4_ToolKit/main.py downloader --classes Dog --type_csv train --limit 200 --noLabels
!python /content/OIDv4_ToolKit/main.py downloader --classes Man --type_csv validation --limit 100 --noLabels
!python /content/OIDv4_ToolKit/main.py downloader --classes Cat --type_csv validation --limit 100 --noLabels
!python /content/OIDv4_ToolKit/main.py downloader --classes Dog --type_csv validation --limit 100 --noLabels
!python /content/OIDv4_ToolKit/main.py downloader --classes Man --type_csv test --limit 100 --noLabels
!python /content/OIDv4_ToolKit/main.py downloader --classes Cat --type_csv test --limit 100 --noLabels
!python /content/OIDv4_ToolKit/main.py downloader --classes Dog --type_csv test --limit 100 --noLabels


		   ___   _____  ______            _    _    
		 .'   `.|_   _||_   _ `.         | |  | |   
		/  .-.  \ | |    | | `. \ _   __ | |__| |_  
		| |   | | | |    | |  | |[ \ [  ]|____   _| 
		\  `-'  /_| |_  _| |_.' / \ \/ /     _| |_  
		 `.___.'|_____||______.'   \__/     |_____|
	

             _____                    _                 _             
            (____ \                  | |               | |            
             _   \ \ ___  _ _ _ ____ | | ___   ____  _ | | ____  ____ 
            | |   | / _ \| | | |  _ \| |/ _ \ / _  |/ || |/ _  )/ ___)
            | |__/ / |_| | | | | | | | | |_| ( ( | ( (_| ( (/ /| |    
            |_____/ \___/ \____|_| |_|_|\___/ \_||_|\____|\____)_|    
                                                          
        
    [INFO] | Downloading Man.
   [ERROR] | Missing the class-descriptions-boxable.csv file.
[DOWNLOAD] | Do you want to download the missing file? [Y/n] y
...145%, 0 MB, 46089 KB/s, 0 seconds passed
[DOWNLOAD] | File clas

In [3]:
# import
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from keras.applications.inception_v3 import preprocess_input

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [14]:
#load pretrained InceptionV3 model and change the top of it
#resize the input, for cheaper and faster training
base_model = InceptionV3(weights='imagenet', include_top=False, classes=3)

#define the top FC layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)

#glue the 2 part together
model = Model(inputs=base_model.input, outputs=predictions)

#check layers-> correct[x]
print("Modified IncpetionV3 layers:")
for i, layer in enumerate(model.layers):
    print(i, layer.name)

Modified IncpetionV3 layers:
0 input_4
1 conv2d_268
2 batch_normalization_267
3 activation_267
4 conv2d_269
5 batch_normalization_268
6 activation_268
7 conv2d_270
8 batch_normalization_269
9 activation_269
10 max_pooling2d_12
11 conv2d_271
12 batch_normalization_270
13 activation_270
14 conv2d_272
15 batch_normalization_271
16 activation_271
17 max_pooling2d_13
18 conv2d_276
19 batch_normalization_275
20 activation_275
21 conv2d_274
22 conv2d_277
23 batch_normalization_273
24 batch_normalization_276
25 activation_273
26 activation_276
27 average_pooling2d_25
28 conv2d_273
29 conv2d_275
30 conv2d_278
31 conv2d_279
32 batch_normalization_272
33 batch_normalization_274
34 batch_normalization_277
35 batch_normalization_278
36 activation_272
37 activation_274
38 activation_277
39 activation_278
40 mixed0
41 conv2d_283
42 batch_normalization_282
43 activation_282
44 conv2d_281
45 conv2d_284
46 batch_normalization_280
47 batch_normalization_283
48 activation_280
49 activation_283
50 average_

In [15]:
# freeze the weights in the first part (CNN) part of the network
for layer in base_model.layers:
    layer.trainable = False

# compile model
model.compile(optimizer='adam', metrics=['accuracy'], loss='categorical_crossentropy')

In [16]:
# work with datagenerator for training

# define pathes of the data
train_dir = '/content/OID/Dataset/train'
validation_dir = '/content/OID/Dataset/validation'
test_dir = '/content/OID/Dataset/test'

#make train, validation, test datagens
train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=15,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=False,
        fill_mode='nearest')
'''
#the original preprocess gave worse accuracy, use the above one
#n inception V3 preprocess (rescale 1./255) included
train_datagen=ImageDataGenerator(
    preprocessing_function=preprocess_input
    )
'''
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

#define some parameters for the training
batch_size = 32
nb_train_samples = 600
nb_validation_samples = 300
nb_test_samples = 300

#generate data for train
train_generator = train_datagen.flow_from_directory(
        train_dir,  # this is the target directory
        target_size=(299,299),  
        batch_size=batch_size,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(299,299),
        batch_size=batch_size,
        class_mode='categorical')

# this is a similar generator, for test data
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(299,299),
        class_mode='categorical')

Found 600 images belonging to 3 classes.
Found 300 images belonging to 3 classes.
Found 300 images belonging to 3 classes.


In [17]:
# 1.TRAIN the FC top through 30 epoch
model.fit(train_generator,
          steps_per_epoch=nb_train_samples // batch_size,
          validation_data=validation_generator,
          validation_steps=nb_validation_samples // batch_size,
          epochs=10
          )

Epoch 1/10
18/18 [==============================] - 35s 2s/step - loss: 1.5488 - accuracy: 0.6004 - val_loss: 0.6502 - val_accuracy: 0.6840
Epoch 2/10
18/18 [==============================] - 27s 2s/step - loss: 0.4098 - accuracy: 0.8433 - val_loss: 0.3288 - val_accuracy: 0.9097
Epoch 3/10
18/18 [==============================] - 27s 2s/step - loss: 0.2697 - accuracy: 0.9155 - val_loss: 0.2348 - val_accuracy: 0.9410
Epoch 4/10
18/18 [==============================] - 28s 2s/step - loss: 0.2429 - accuracy: 0.9085 - val_loss: 0.2497 - val_accuracy: 0.9236
Epoch 5/10
18/18 [==============================] - 27s 2s/step - loss: 0.1624 - accuracy: 0.9507 - val_loss: 0.2575 - val_accuracy: 0.9097
Epoch 6/10
18/18 [==============================] - 27s 2s/step - loss: 0.1496 - accuracy: 0.9542 - val_loss: 0.3073 - val_accuracy: 0.8681
Epoch 7/10
18/18 [==============================] - 27s 2s/step - loss: 0.1838 - accuracy: 0.9331 - val_loss: 0.1874 - val_accuracy: 0.9410
Epoch 8/10
18/18 [==

In [23]:
# 2.TRAIN freeze the first parts of the network
for layer in model.layers[:172]:
    layer.trainable = False

for layer in model.layers[172:]:
    layer.trainable = True

model.compile(optimizer=SGD(learning_rate=0.00001,momentum=0.95), metrics=['accuracy'], loss='categorical_crossentropy')

In [24]:
# train the network from the 172th layer (including CNN and FC layers as well)
model.fit(train_generator,
          steps_per_epoch=nb_train_samples // batch_size,
          validation_data=validation_generator,
          validation_steps=nb_validation_samples // batch_size,
          epochs=10
          )

Epoch 1/10
18/18 [==============================] - 36s 2s/step - loss: 0.0782 - accuracy: 0.9806 - val_loss: 0.2283 - val_accuracy: 0.9375
Epoch 2/10
18/18 [==============================] - 28s 2s/step - loss: 0.0918 - accuracy: 0.9754 - val_loss: 0.2216 - val_accuracy: 0.9444
Epoch 3/10
18/18 [==============================] - 28s 2s/step - loss: 0.0890 - accuracy: 0.9736 - val_loss: 0.2106 - val_accuracy: 0.9514
Epoch 4/10
18/18 [==============================] - 28s 2s/step - loss: 0.0610 - accuracy: 0.9824 - val_loss: 0.2002 - val_accuracy: 0.9549
Epoch 5/10
18/18 [==============================] - 28s 2s/step - loss: 0.0832 - accuracy: 0.9754 - val_loss: 0.2010 - val_accuracy: 0.9549
Epoch 6/10
18/18 [==============================] - 28s 2s/step - loss: 0.0872 - accuracy: 0.9718 - val_loss: 0.2037 - val_accuracy: 0.9514
Epoch 7/10
18/18 [==============================] - 28s 2s/step - loss: 0.0761 - accuracy: 0.9789 - val_loss: 0.2014 - val_accuracy: 0.9514
Epoch 8/10
18/18 [==

In [25]:
# measure test accuracy
print("Test accuracy:")
acc = model.evaluate(test_generator)

Test accuracy:
10/10 [==============================] - 6s 535ms/step - loss: 0.2019 - accuracy: 0.9267
